In [1]:
# Initialize Earth Engine
import ee
import geemap
ee.Initialize()

# Create map
Map = geemap.Map(center=[23.6850, 90.3563], zoom=7)
bangladesh = ee.FeatureCollection("FAO/GAUL/2015/level0") \
              .filter(ee.Filter.eq('ADM0_NAME', 'Bangladesh'))
Map.addLayer(bangladesh, {}, 'Bangladesh')

# Improved Landsat data function with proper cloud masking
def get_landsat_data(start_year, end_year):
    # Landsat 5 (1984-2011)
    if end_year <= 2011:
        collection = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
        ndvi_bands = ['SR_B4', 'SR_B3']
        rgb_bands = ['SR_B3', 'SR_B2', 'SR_B1']
    # Landsat 7 (1999-2022)
    elif start_year <= 2013:
        collection = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
        ndvi_bands = ['SR_B4', 'SR_B3']
        rgb_bands = ['SR_B3', 'SR_B2', 'SR_B1']
    # Landsat 8/9 (2013-present)
    else:
        collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
        ndvi_bands = ['SR_B5', 'SR_B4']
        rgb_bands = ['SR_B4', 'SR_B3', 'SR_B2']
    
    # Filter by date and location
    collection = collection.filterBounds(bangladesh) \
                         .filterDate(f'{start_year}-01-01', f'{end_year}-12-31')
    
    # Function to apply scaling and cloud masking
    def process_image(img):
        # Apply scaling to optical bands only
        optical = img.select('SR_B.').multiply(0.0000275).add(-0.2)
        # Get QA_PIXEL band (unscaled)
        qa = img.select('QA_PIXEL')
        # Create cloud mask (bit 3 is cloud)
        cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)
        # Apply mask and return with original bands
        return optical.updateMask(cloud_mask).copyProperties(img, ['system:time_start'])
    
    # Process and reduce collection
    return collection.map(process_image).median(), ndvi_bands, rgb_bands

# Process all time periods
time_periods = [
    (2006, 2010),
    (2011, 2015), 
    (2016, 2020),
    (2021, 2023)  # Note: 2025 data may not be complete
]

# Visualization parameters
ndvi_params = {'min': -0.2, 'max': 0.8, 'palette': ['red', 'yellow', 'green']}
rgb_params = {'min': 0, 'max': 0.3}

for start_year, end_year in time_periods:
    label = f"{start_year}-{end_year}"
    print(f"Processing {label}...")
    
    try:
        # Get data
        image, ndvi_bands, rgb_bands = get_landsat_data(start_year, end_year)
        
        # Calculate NDVI
        ndvi = image.normalizedDifference(ndvi_bands).rename('NDVI')
        image = image.addBands(ndvi)
        
        # Add to map
        Map.addLayer(
            image.clip(bangladesh).select(rgb_bands),
            rgb_params,
            f'Landsat {label} (RGB)'
        )
        Map.addLayer(
            image.select('NDVI').clip(bangladesh),
            ndvi_params,
            f'NDVI {label}'
        )
        
        # Export
        geemap.ee_export_image(
            image.select('NDVI').clip(bangladesh),
            filename=f'ndvi_{label}.png',
            scale=500
        )
    except Exception as e:
        print(f"Error processing {label}: {str(e)}")

# Add layer control and display
Map.addLayerControl()
Map

Processing 2006-2010...
The filename must end with .tif
Processing 2011-2015...
The filename must end with .tif
Processing 2016-2020...
The filename must end with .tif
Processing 2021-2023...
The filename must end with .tif


Map(center=[23.685, 90.3563], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprig…